# Problem - "Positive Volume Ladder"
Rank stocks in 2015 by Positive Volume in ascending order.
`Positive Volume(t) = Volume(t) if Adj.Close(t) >= Adj.Close(t-1) else 0`

In [4]:
%run notebooks/utils.ipynb

## Solution #1
For stocks in `bars_1` table

In [5]:
execute("""
WITH data AS (
    SELECT symbol,
        CASE WHEN adj_close >= LAG(adj_close) OVER (PARTITION BY symbol ORDER BY "date") THEN volume ELSE 0 END
        AS positive_volume
    FROM bars_1 WHERE EXTRACT(YEAR FROM "date") = 2015
)
SELECT symbol, RANK() OVER (ORDER BY SUM(positive_volume)) AS positive_volume_rank FROM data GROUP BY symbol
""")

,symbol,positive_volume_rank
0,AMCR,1
1,BIO,2
2,CZR,3
3,TECH,4
4,TDY,5
...,...,...
181,TSLA,182
182,T,183
183,BAC,184
184,AMZN,185


## Solution #2
For all stocks (across `bars_1` and `bars_2` tables)

In [6]:
execute("""
WITH data AS (
    SELECT symbol,
        CASE WHEN adj_close >= LAG(adj_close) OVER (PARTITION BY symbol ORDER BY "date") THEN volume ELSE 0 END
        AS positive_volume
        FROM (
            SELECT symbol, "date", adj_close, volume FROM bars_1
            UNION
            SELECT symbol, "date", adj_close, volume FROM bars_2
        ) AS bars
        WHERE EXTRACT(YEAR FROM "date") = 2015
)
SELECT symbol, RANK() OVER (ORDER BY SUM(positive_volume)) AS positive_volume_rank FROM data GROUP BY symbol
""")

,symbol,positive_volume_rank
0,AMCR,1
1,BIO,2
2,CZR,3
3,TDY,4
4,TECH,5
...,...,...
181,TSLA,182
182,T,183
183,BAC,184
184,AMZN,185
